In [37]:
import os
import boto3
import sagemaker
import sagemaker_pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from sagemaker import get_execution_role
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [3]:
role = get_execution_role()
jars = sagemaker_pyspark.classpath_jars()
classpath = ":".join(sagemaker_pyspark.classpath_jars())
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .master("local[*]")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/14 18:26:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
cn_regions = ["cn-north-1", "cn-northwest-1"]
region = boto3.Session().region_name
endpoint_domain = "com.cn" if region in cn_regions else "com"
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.endpoint", "s3.{}.amazonaws.{}".format(region, endpoint_domain)
)

In [27]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .load("s3a://recsys-aws/silver_data/run-1691792595139-part-r-00001".format(region))
)

In [28]:
user_indexer = StringIndexer(inputCol="userid", outputCol="userid_index")
df = user_indexer.fit(df).transform(df)

product_indexer = StringIndexer(inputCol="productid", outputCol="productid_index")
df = product_indexer.fit(df).transform(df)

df = df.withColumn("rating", col("rating").cast("int"))
df = df.withColumn("userid_index", col("userid_index").cast("int"))
df = df.withColumn("productid_index", col("productid_index").cast("int"))

In [29]:
df.show(5)

23/08/14 18:43:43 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


+--------------+----------+------+------------+---------------+
|        userid| productid|rating|userid_index|productid_index|
+--------------+----------+------+------------+---------------+
|A1LVOG39LLLJGB|B00889VG6W|     4|       21651|           2389|
| AO0SD1BIIZQ18|B00889VG6W|     5|       16198|           2389|
|A1GIKW162IACAL|B00889VG6W|     1|       74811|           2389|
|A14EQN2XIZ6JTV|B00889VG6W|     5|       54195|           2389|
|A3IPJS7MGS03ES|B00889VG6W|     4|      199806|           2389|
+--------------+----------+------+------------+---------------+
only showing top 5 rows



In [30]:
df.describe().show()

23/08/14 18:43:49 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:44:16 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB
+-------+--------------------+----------+------------------+------------------+------------------+
|summary|              userid| productid|            rating|      userid_index|   productid_index|
+-------+--------------------+----------+------------------+------------------+------------------+
|  count|              376032|    376032|            376032|            376032|            376032|
|   mean|                null|      null| 4.127108863075483|105033.70971087567|11069.678394391967|
| stddev|                null|      null|1.3043246790624412| 88251.89756084904|17114.201043419223|
|    min|A0002574WYJMBWKNCPY8|B00889VG6W|                 1|                 0|                 0|
|    max|       AZZZU2TD7Q3ET|B00LU0LTOU|                 5|            276442|             73778|
+-------+--------------------+----------+------------------+------------------+------------------+



In [31]:
train, test = df.randomSplit([0.8, 0.2])

In [32]:
recommender = ALS(userCol='userid_index', ratingCol='rating', itemCol='productid_index')
recommender = recommender.fit(train)

23/08/14 18:44:30 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:44:34 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:44:38 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:44:41 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:44:49 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


...

23/08/14 18:46:41 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:46:47 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:46:52 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:46:57 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:47:04 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


In [33]:
preds = recommender.transform(test)

In [35]:
preds = preds.fillna({"prediction": 3})

In [36]:
preds.show(5)

23/08/14 18:52:27 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:52:29 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:52:30 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:52:42 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


+--------------------+----------+------+------------+---------------+----------+
|              userid| productid|rating|userid_index|productid_index|prediction|
+--------------------+----------+------+------------+---------------+----------+
|A00728291XHL0K3ET...|B008FX4LO0|     5|        5223|           1824| 1.2663755|
|A01058772XBEXI92G...|B00H997IO6|     4|       46416|           3059|       3.0|
|A0140494QSPWAFGBI083|B00FQ0U57M|     5|       46435|          32993|       3.0|
|A00732952F8EWGDMD...|B009GAOHQ8|     5|       46399|          25097|       3.0|
|A00473363TJ8YSZ3Y...|B00AE07JIM|     4|        8663|           2543| 0.7306286|
+--------------------+----------+------+------------+---------------+----------+
only showing top 5 rows



In [38]:
evaluator = RegressionEvaluator(labelCol='rating')
evaluator.evaluate(preds)

23/08/14 18:53:25 WARN DAGScheduler: Broadcasting large task binary with size 12.4 MiB


23/08/14 18:53:26 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:53:28 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:53:40 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


23/08/14 18:53:43 WARN DAGScheduler: Broadcasting large task binary with size 12.5 MiB


2.4538526017862647